In [1]:
import os
import gc
import numpy as np
import json
import torch
from torch.utils.data import DataLoader
from torchinfo import summary

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from src.utils import *
from src.runner import run_config

from src.models import UNet, UNet2, UNet4, UNet8, UNetShallow, UNet2Shallow, UNet4Shallow, UNet8Shallow
from src.model_parts import SingleConv, DoubleConv, TripleConv

c:\Users\wittd\anaconda3\Lib\site-packages\pytools\persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


Cupy implementation is not available. Make sure you have the right version of Cupy and CUDA installed.
Optional dependecy Dask_image is not installed. Implementations using it will be ignored.


c:\Users\wittd\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


tiling_rate = 4
overlap_rate = 0.25

tiling_rate = 8
overlap_rate = 0.3 vagy 0.2

tiling_rate = 10
overlap_rate = 0.25

tiling_rate = 16
overlap_rate = 0.2

In [2]:
# out of memory error
config = {
'data_path': 'C:/7_felev/data',
    'save_path': 'C:/7_felev/szakdoga',
    # Each run name should be unique, even if the same model is used 
    # or it will overwrite the previous run
    'run_name': 'UNetSh8_tile8_len8_singleconv',
    'project_name': 'Tile8',
    'wandb_logging': True,

    # Unet, UNet2, UNet4, UNet8, UNetShallow, Unet2Shallow, UNet4Shallow, UNet8Shallow
    'model': UNet8Shallow,
    'down_conv': SingleConv,
    'up_conv': SingleConv,
    'bilinear': False,

    'biosensor_length': 8,
    'input_scale': 1,
    # for the imput scaling
    'SRRF_mode': 'eSRRF', # 'eSRRF', 'SRRF', None
    'mask_scale': 8,
    
    'batch_size': 13,
    'epochs': 20,
    'learning_rate': 0.01,
    # K-fold cross validation
    'k': 5,

    'mask_type': bool,
    'augment': True,
    'normalize': True,
    'shuffle': True,
    
    'tiling': True,
    'tiling_ratio': 10,
    # tile size = (80 * input_scale) / tiling_ratio
    'overlap_rate': 0.25,

    'dilation': 0,
}

config6 = config.copy()
config6['run_name'] = 'UNetSh8_tile8_len8_doubleconv'
config6['down_conv'] = config6['up_conv'] = DoubleConv

In [3]:
config = {
'data_path': 'C:/7_felev/data',
    'save_path': 'C:/7_felev/szakdoga',
    # Each run name should be unique, even if the same model is used 
    # or it will overwrite the previous run
    'run_name': 'UNetSh_tile5_len8_doubleconv',
    'project_name': 'Tile5',
    'wandb_logging': True,

    # Unet, UNet2, UNet4, UNet8, UNetShallow, Unet2Shallow, UNet4Shallow, UNet8Shallow
    'model': UNetShallow,
    'down_conv': DoubleConv,
    'up_conv': DoubleConv,
    'bilinear': False,

    'biosensor_length': 8,
    'input_scale': 1,
    # for the imput scaling
    'SRRF_mode': 'eSRRF', # 'eSRRF', 'SRRF', None
    'mask_scale': 1,
    
    'batch_size': 13,
    'epochs': 20,
    'learning_rate': 0.01,
    # K-fold cross validation
    'k': 5,

    'mask_type': bool,
    'augment': True,
    'normalize': True,
    'shuffle': True,
    
    'tiling': True,
    'tiling_ratio': 16,
    # tile size = (80 * input_scale) / tiling_ratio
    'overlap_rate': 0.2,

    'dilation': 0,
}

test_tiling(config)
run_config(config)

Tiling test
Bio tiles: torch.Size([361, 8, 5, 5]), Mask tiles: torch.Size([361, 5, 5])

Using device cuda
Datasets created!
Starting run 1...
UNetShallow


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: wdavid (wdave). Use `wandb login --relogin` to force relogin


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.6887989044189453, Detection rate: 0.20990498128419235


Epoch 1/20: 100%|██████████| 104/104 [00:36<00:00,  2.88img/s, loss (batch)=1.07]


...

Epoch 20/20: 100%|██████████| 104/104 [00:28<00:00,  3.61img/s, loss (batch)=0.7]


Checkpoint 20 saved!


detection_rate,▁▅▅▄█▅▄▄▅▃▄▄▄▃▂▅▄▃▅▃
epoch,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇█
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇█████
train loss,█▄▄▄▄▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▂▂▁▁▁▁▂▂▁▁▁▁▂▁▂▂▁▁▁▁
validation Dice,▅▄▅▆▁▆▇▇▇▇▇▇██▇▆▇███
detection_rate,0.88602
epoch,20
learning rate,0.01
step,160
train loss,0.69994




Best epoch: 20


Test metrics: {
    "dice_score": 0.7282297611236572,
    "detection_rate": 0.7999567941240009,
    "avg_pred_cell_size": 1261.8147448015122,
    "std_pred_cell_size": 707.9146527135755,
    "avg_mask_cell_size": 1078.3916612659323,
    "std_mask_cell_size": 635.7772260178464
}
Run 5 finished!
All runs finished!


In [4]:
config2 = config.copy()
config2['run_name'] = 'UNetSh_tile5_len8_singleconv'
config2['down_conv'] = config2['up_conv'] = SingleConv

test_tiling(config2)
run_config(config2)

Tiling test
Bio tiles: torch.Size([361, 8, 5, 5]), Mask tiles: torch.Size([361, 5, 5])

Using device cuda
Datasets created!
Starting run 1...
UNetShallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.6894967555999756, Detection rate: 1.0526921969478837


Epoch 1/20: 100%|██████████| 104/104 [00:32<00:00,  3.20img/s, loss (batch)=1.02]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:27<00:00,  3.73img/s, loss (batch)=0.726]


Checkpoint 20 saved!


detection_rate,▁▄▆▅▆▇█▆▆▆▆▇█▇█▇▆▇▆▇
epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇████
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇██
train loss,▇▅▅█▅▃▅▃▄▃▄▃▂▃▂▁▂▂▂▂▂▃▃▁▂▄▂▃▁▁▂▂▂▂▁▃▂▂▂▂
validation Dice,▁▆▆▆▆▇▅▇▅▆▇▆▂▇▄▆▇▇██
detection_rate,0.96118
epoch,20
learning rate,0.01
step,160
train loss,0.726




Best epoch: 20


Test metrics: {
    "dice_score": 0.7106262445449829,
    "detection_rate": 0.8695182544826097,
    "avg_pred_cell_size": 1382.9192546583852,
    "std_pred_cell_size": 802.6277803120429,
    "avg_mask_cell_size": 1078.3916612659323,
    "std_mask_cell_size": 635.7772260178464
}
Run 5 finished!
All runs finished!


In [5]:
config3 = config.copy()
config3['run_name'] = 'UNetSh4_tile5_len8_doubleconv'
config3['down_conv'] = config3['up_conv'] = DoubleConv
config3['model'] = UNet4Shallow
config3['mask_scale'] = 4

test_tiling(config3)
run_config(config3)

Tiling test
Bio tiles: torch.Size([361, 8, 5, 5]), Mask tiles: torch.Size([361, 20, 20])

Using device cuda
Datasets created!
Starting run 1...
UNet4Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.6545919179916382, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:30<00:00,  3.43img/s, loss (batch)=1.1]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:29<00:00,  3.51img/s, loss (batch)=0.725]


Checkpoint 20 saved!


detection_rate,▁▅▂▇▇▄▆█▅▄▆▄▄▄▄▄▅▄▄▄
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
learning rate,███████████▂▂▂▂▂▂▂▁▁
step,▁▁▁▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████
train loss,█▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▂▂▁▁▂▂▁▁▂▁▂▂▂▁
validation Dice,▆▂▆▁▂█▅▃▇█▆█████████
detection_rate,0.68867
epoch,20
learning rate,0.0001
step,160
train loss,0.72492




Best epoch: 13


Test metrics: {
    "dice_score": 0.6774815917015076,
    "detection_rate": 0.5652994679938219,
    "avg_pred_cell_size": 871.5775842440802,
    "std_pred_cell_size": 682.1189905392592,
    "avg_mask_cell_size": 500.6998669984555,
    "std_mask_cell_size": 460.70972490830883
}
Run 5 finished!
All runs finished!


In [6]:
config4 = config3.copy()
config4['run_name'] = 'UNetSh4_tile5_len8_singleconv'
config4['down_conv'] = config4['up_conv'] = SingleConv

test_tiling(config4)
run_config(config4)

Tiling test
Bio tiles: torch.Size([361, 8, 5, 5]), Mask tiles: torch.Size([361, 20, 20])

Using device cuda
Datasets created!
Starting run 1...
UNet4Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.6545919179916382, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:28<00:00,  3.61img/s, loss (batch)=1.08]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:27<00:00,  3.80img/s, loss (batch)=0.761]


Checkpoint 20 saved!


detection_rate,▁▁█▆▅▆▇▆▅▆▆▅▆▆▇▅▆▆▆▆
epoch,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇████
train loss,█▆▆▆▆▄▃▂▂▂▄▂▂▃▁▃▂▃▂▃▂▂▂▃▂▃▂▂▂▁▂▂▂▂▂▂▂▂▁▁
validation Dice,▆▆▁▇▇█▇▇██████▇█████
detection_rate,0.70197
epoch,20
learning rate,0.01
step,160
train loss,0.76078




Best epoch: 20


Test metrics: {
    "dice_score": 0.6656543016433716,
    "detection_rate": 0.6722155483095933,
    "avg_pred_cell_size": 928.7839864692367,
    "std_pred_cell_size": 808.9712831405486,
    "avg_mask_cell_size": 500.6998669984555,
    "std_mask_cell_size": 460.70972490830883
}
Run 5 finished!
All runs finished!


In [7]:
config5 = config.copy()
config5['run_name'] = 'UNetSh8_tile5_len8_singleconv'
config5['down_conv'] = config5['up_conv'] = SingleConv
config5['model'] = UNet8Shallow
config5['mask_scale'] = 8

test_tiling(config5)
run_config(config5)

Tiling test
Bio tiles: torch.Size([361, 8, 5, 5]), Mask tiles: torch.Size([361, 40, 40])

Using device cuda
Datasets created!
Starting run 1...
UNet8Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.6439378261566162, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:31<00:00,  3.29img/s, loss (batch)=1.08]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:30<00:00,  3.38img/s, loss (batch)=0.826]


Checkpoint 20 saved!


detection_rate,▁▁▁▁▁▁▁▁▁▁▁▇▆▇▇█▇███
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
learning rate,██████▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▇▇▇▇▇█████
train loss,█▅▄▅▄▃▃▃▂▂▂▃▂▂▂▁▂▂▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
validation Dice,▁▁▁▁▁▁▁▁▁▁▁▇▇▇█▆█▆▇█
detection_rate,0.73003
epoch,20
learning rate,0.001
step,160
train loss,0.82631




Best epoch: 20


Test metrics: {
    "dice_score": 0.6612011790275574,
    "detection_rate": 0.6698236446766819,
    "avg_pred_cell_size": 1011.0588203924915,
    "std_pred_cell_size": 846.7234067546142,
    "avg_mask_cell_size": 476.0359471954605,
    "std_mask_cell_size": 454.3744423582587
}
Run 5 finished!
All runs finished!


In [8]:
config6 = config5.copy()
config6['run_name'] = 'UNetSh8_tile5_len8_doubleconv'
config6['down_conv'] = config6['up_conv'] = DoubleConv

test_tiling(config6)
run_config(config6)

Tiling test
Bio tiles: torch.Size([361, 8, 5, 5]), Mask tiles: torch.Size([361, 40, 40])

Using device cuda
Datasets created!
Starting run 1...
UNet8Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.6439378261566162, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:35<00:00,  2.90img/s, loss (batch)=1.14]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:28<00:00,  3.59img/s, loss (batch)=0.715]


Checkpoint 20 saved!


detection_rate,▁▇█▅▅▆▆▄▅▃▆▄▅▆▄▅▄▅▅▅
epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇████
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train loss,█▇▅▄▃▃▃▃▂▃▂▃▂▃▃▂▁▂▂▂▂▂▂▃▂▂▂▃▂▃▂▂▂▃▂▃▂▃▂▂
validation Dice,▆▂▁▇▇▇▇█▇▇▇▇█▇██▇███
detection_rate,0.66457
epoch,20
learning rate,0.01
step,160
train loss,0.71541




Best epoch: 19


Test metrics: {
    "dice_score": 0.6689872741699219,
    "detection_rate": 0.5865447419986937,
    "avg_pred_cell_size": 892.7499478006681,
    "std_pred_cell_size": 767.704525994733,
    "avg_mask_cell_size": 476.0359471954605,
    "std_mask_cell_size": 454.3744423582587
}
Run 5 finished!
All runs finished!
